## Oppstart

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
import dhlab.nbtext as nb
import dhlab.token_map as tm
from dhlab.nbtext import get_urn, make_graph, get_urn, relaterte_ord, navn, totals
import dhlab.graph_networkx_louvain as gnl
import pandas as pd
import networkx as nx

%matplotlib inline

plot = lambda x,  lw=3, alpha= 0.7: x.plot(figsize=(15,6), lw=lw, alpha=alpha)


In [18]:
from dhlab.module_update import update, css
css()

# Hjelpekommandoer

I den her notebooken er det satt opp kode for å telle opp navn i et korpus. Både listen med navn og korpuset må foreligge som excelfiler. Noen av kommandoene vil bli lagt til `token_map`-modulen 

* `tell_navn` teller opp navnene på tvers av korpuset 
* `korpus_oppføring` finner data i korpuset
* `vis_bok` viser navn i en spesifikk bok (benytter oppførings-kommandoen)
* `aggregate_pr_col` aggregerer navnedata over kolonner.

In [19]:
import pandas as pd
from nbtext import metadata

def alle_korpusnavn(korpus):
    urner = nb.pure_urn(korpus)
    alle_navn = tm.combine_names(tm.corpus_names(urner))
    return alle_navn

def tell_navn(korpus, token_map):
    res = dict()
    for urn in nb.pure_urn(korpus['urn']):
        try:
            res[urn] = tm.count_name_strings(str(urn), token_map)
            res[urn].columns = [str(urn)]
        except:
            print("Problemer med å telle fra: ", urn)
    return pd.concat([res[u] for u in res], axis=1)

def korpus_oppføring(df, text, column='tittel'):
    return nb.pure_urn(df[df[column].str.contains(text)]['urn'])

def add_metadata(korpus, how='inner'):
    f = nb.frame([(z[0], '_'.join([str(x) for x in z[2:4]])) for z in nb.metadata(korpus['urn'])], 'urn metadata'.split())
    f = f.astype({'urn':'str'})
    return korpus.merge(f, on = 'urn', how = how)
    
vis_bok = lambda x, column = 'tittel': nb.frame_sort(nb.frame(opptelling[korpus_oppføring(korpus, x, column)[0]].dropna(), nb.metadata(korpus_oppføring(korpus, x, column)[0])[0][3]))

    

def bygg_allgraf(korpus):
    graphs = dict()
    for u in korpus.urn:
        graphs[u] = tm.character_network(u, redigerte_navn)
    df_graphs = [nx.to_pandas_adjacency(graphs[x]) for x in graphs]
    g = df_graphs[0]
    for z in df_graphs[1:]:
        g = g.add(z, fill_value=0).fillna(0)

    total_graph = nx.from_pandas_adjacency(g)
    return total_graph

## Navn i bøker


Vi søker etter navn i en bok, og en serie av bøker.

Kommandoene er:
1. `names` henter et sett med forslag til navn, fra nbtext
1. `show_names` gir en oversikt over funnene, fra token_map
1. `names_to_token_map_file` lagrer navneforslagene til en excelfil (eller csv) for redigering (token_map)
1. `read_token_map_file` henter redigert fil tilbake for analyse (token_map)
1. `character_network` lager graf for navnene (token_map)
1. `show_graph`  tegner grafen (fra graph_networkx_louvain)
1. `show_communities` viser clustre i grafen (graph_networkx_louvain)



## Korpus

Henter korpus fra excelfil. Det er korpuset det skal telles opp fra.

In [20]:
korpus = nb.restore_metadata_from_excel('Brev, Hamsunskorpus.xlsx')
korpus

,urn,forf,år,tittel,undertittel,forlag,sjanger,sesamid,Unnamed: 8
0,2008091204050,"Hamsun, Knut",1997,Knut Hamsuns brev,NaN,"Gyldendal;Oslo : Gyldendal, 1994-2000",fiction,016db7c52214bce8e3612c2608b4b61c,nob
1,2010031603013,"Hamsun, Knut",1995,Knut Hamsuns brev,NaN,"Gyldendal;Oslo : Gyldendal, 1994-2000",fiction,23df19922b203f588261e34304351de1,nob
2,2009032400042,"Hamsun, Knut",1999,Knut Hamsuns brev,NaN,"Gyldendal;Oslo : Gyldendal, 1994-2000",fiction,3be2e2a46b0a788081c6e72cbf24761f,nob
3,2008010400002,"Hamsun, Knut",1994,Knut Hamsuns brev,NaN,"Gyldendal;Oslo : Gyldendal, 1994-2000",fiction,131e85b5bbc3800dcb60bc9afd588459,nob
4,2008112804029,"Hamsun, Knut",2000,Knut Hamsuns brev,NaN,"Gyldendal;Oslo : Gyldendal, 1994-2000",fiction,8206fe4a8ec2fac45ac6f3d2a9c7639c,mul
5,2008110500029,"Hamsun, Knut",2001,Knut Hamsuns brev,NaN,"Gyldendal;Oslo : Gyldendal, 1994-2000",fiction,33b6512367fc496d55846e601217d98d,nob


## Konkordanssøk



In [24]:
nb.urn_concordance(word = 'Sarpsborg', urns = korpus) 

"Knut Hamsuns brev, Hamsun, Knut, 1995",. - Rejste altsaa til,Sarpsborg,og tilbage til Kristiania .
"Knut Hamsuns brev, Hamsun, Knut, 1995",Nogle Dage senere atter til,Sarpsborg,. Bare Sagf ørergrejer forresten
"Knut Hamsuns brev, Hamsun, Knut, 1999","til november 1890 , i",Sarpsborg,fra juni til september 1891
"Knut Hamsuns brev, Hamsun, Knut, 1994",] Min fremtidige Adresse er,Sarpsborg,", Norwegen"
"Knut Hamsuns brev, Hamsun, Knut, 1994",[ ? Kristiania ?,Sarpsborg,? mai-juni 71891 j 1
"Knut Hamsuns brev, Hamsun, Knut, 1994",None,Sarpsborg,", 30 Maj . ["
"Knut Hamsuns brev, Hamsun, Knut, 1994",[,Sarpsborg,10.6.1891 ]
"Knut Hamsuns brev, Hamsun, Knut, 1994",None,Sarpsborg,", Norge , 10 Juni"
"Knut Hamsuns brev, Hamsun, Knut, 2001",og som han fortsatte i,Sarpsborg,"( sommeren , høsten 1891"
"Knut Hamsuns brev, Hamsun, Knut, 2001",inntrykk fra sitt ophold i,Sarpsborg,i 90 - årene »
"Knut Hamsuns brev, Hamsun, Knut, 2001",Da Knut Hamsun bodde i,Sarpsborg,"» , en samling småintervjuer"


### Hent redigert fil

Husk at verdien på orient må være lik mellom lagring og lesing av excelfil. Typisk for den her type redigering er radorientering. Excel har en begrensning på 256 kolonner, som blir for smått for de fleste anvendelser. Det er tilnærmet ubegrenset kapasitet på antall rader.

In [7]:
redigerte_navn = tm.read_token_map_file('samledestedsnavn_Hamsunkorpus.xls', orient = "row")

## Tell opp navn med hensyn til det som står i `redigerte_navn`

Bruker hjelpekommando `tell_navn`. Det kan ta litt tid å telle opp, et par minutter.

In [8]:
opptelling = tell_navn(korpus, redigerte_navn)

Sånn ser opptellingen ut, en kolonne pr. bok. 

In [9]:
opptelling.head()

,2008091204050,2010031603013,2009032400042,2008010400002,2008112804029,2008110500029
Aas,6.0,71.0,2.0,18.0,1.0,20.0
Afrika,NaN,2.0,NaN,NaN,6.0,NaN
Amerika,39.0,27.0,66.0,133.0,50.0,26.0
Argentina,NaN,NaN,1.0,NaN,1.0,NaN
Asien,1.0,3.0,NaN,NaN,NaN,1.0


Opptelling totalt for korpuset, summering over alle bøkene.

In [14]:
nb.frame_sort(nb.frame(opptelling.sum(axis=1)))

,0
Christiania,1812.0
Nørholmen,1247.0
Norge,738.0
Grimstad,682.0
København,556.0
Tyskland,487.0
Paris,359.0
Verden,342.0
Amerika,341.0
Hamarøy,327.0


In [11]:
steder = nb.frame_sort(nb.frame(opptelling.sum(axis=1)))
steder.to_dict()[0]

{'Aas': 118.0,
 'Afrika': 8.0,
 'Amerika': 341.0,
 'Argentina': 2.0,
 'Asien': 5.0,
 'Asker': 29.0,
 'Atlanta': 1.0,
 'Atlanterhavet': 11.0,
 'Aurdal': 147.0,
 'Australien': 6.0,
 'Baku': 6.0,
 'Balberg': 18.0,
 'Batum': 1.0,
 'Bayern': 4.0,
 'Belgien': 17.0,
 'Bergen': 259.0,
 'Berlin': 204.0,
 'Bodø': 61.0,
 'Boston': 6.0,
 'Brasil': 2.0,
 'Brevik': 5.0,
 'Byen': 162.0,
 'Bygdø': 27.0,
 'California': 3.0,
 'Chicago': 38.0,
 'Christiania': 1812.0,
 'Dakota': 6.0,
 'Danmark': 169.0,
 'Drammen': 26.0,
 'Drøbak': 82.0,
 'Dubrovnik': 19.0,
 'Egersund': 18.0,
 'Egypten': 4.0,
 'Elverum': 21.0,
 'England': 101.0,
 'Europa': 50.0,
 'Finland': 79.0,
 'Finmarken': 7.0,
 'Fjell': 16.0,
 'Frankrike': 83.0,
 'Fredrikstad': 9.0,
 'Gaarden': 159.0,
 'Gaden': 56.0,
 'Garmo': 16.0,
 'Gaustad': 2.0,
 'Gjørvik': 6.0,
 'Gjøvik': 48.0,
 'Grand_Hotel': 58.0,
 'Grekenland': 1.0,
 'Grimstad': 682.0,
 'Grønland': 26.0,
 'Gudbrandsdalen': 13.0,
 'Haag': 7.0,
 'Hamar': 16.0,
 'Hamarøy': 327.0,
 'Hammerfest': 2

# Bygger grafer for alle bøkene

Benytt hjelpekommandoen `bygg_allgraf` for å samle grafene i en stor total graf.

In [12]:
korpusgraf = bygg_allgraf(korpus)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Visualiser og analyser på vanlig måte.

In [ ]:
gnl.show_graph(korpusgraf, spread=2.3)

Forslag til clustre basert på grafen. Clusteralgoritmen for Louvain kan synes å være litt random, så resultatene kan varierere mellom kjøringer. Men det ser skapelig ut, og gir mening.

In [ ]:
gnl.show_communities(korpusgraf)